#Q1: Some of the facilities charge a fee to members, but some do not.
#Write a SQL query to produce a list of the names of the facilities that do. */

SELECT facid 
FROM `Facilities` 
WHERE membercost>0

#Q2: How many facilities do not charge a fee to members? */

SELECT count(facid) 
FROM `Facilities` 
WHERE membercost>0

#Q3: Write an SQL query to show a list of facilities that charge a fee to members,
#where the fee is less than 20% of the facility's monthly maintenance cost.
#Return the facid, facility name, member cost, and monthly maintenance of the
#facilities in question. */

SELECT facid, name, membercost, monthlymaintenance 
FROM `Facilities` 
WHERE membercost>0  and membercost<(.2 * monthlymaintenance)

#Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
#Try writing the query without using the OR operator. */

SELECT facid, name, membercost, guestcost, initialoutlay, monthlymaintenance 
FROM Facilities 
WHERE facid in (1, 5) and facid not in (2,4)

#Q5: Produce a list of facilities, with each labelled as
#'cheap' or 'expensive', depending on if their monthly maintenance cost is
#more than $100. Return the name and monthly maintenance of the facilities
#in question. */

SELECT name, 
	Case when monthlymaintenance > 100 then 'expensive' 
	when monthlymaintenance < 100 then 'cheap' end as label
FROM Facilities

#Q6: You'd like to get the first and last name of the last member(s)
#who signed up. Try not to use the LIMIT clause for your solution. */

SELECT surname, firstname, joindate
FROM `Members`
Order by joindate desc

#Q7: Produce a list of all members who have used a tennis court.
#Include in your output the name of the court, and the name of the member
#formatted as a single column. Ensure no duplicate data, and order by
#the member name. */

SELECT m.firstname, m.surname as name, f.name
FROM Members as m
left join Bookings as b
on m.memid = b.memid
left join Facilities as f
on f.facid = b.facid
where b.facid = 0 or b.facid=1
Order by m.surname

#Q8: Produce a list of bookings on the day of 2012-09-14 which
#will cost the member (or guest) more than $30. Remember that guests have
#different costs to members (the listed costs are per half-hour 'slot'), and
#the guest user's ID is always 0. Include in your output the name of the
#facility, the name of the member formatted as a single column, and the cost.
#Order by descending cost, and do not use any subqueries. */

SELECT f.name as facility_name, CONCAT_WS(firstname, ' ', surname),
CASE WHEN b.memid = 0 THEN b.slots*f.guestcost 
ELSE b.slots*f.membercost END AS cost 
FROM Bookings as b
left join Facilities as f
on b.facid = f.facid
left join Members as m
on m.memid = b.memid
WHERE b.starttime like '2012-09-14%' and ((b.memid = 0 AND b.slots*f.guestcost > 30) OR (b.memid>0 AND b.slots*f.membercost >30))
Order by cost desc

#Q9: This time, produce the same result as in Q8, but using a subquery. */

SELECT *
FROM (SELECT f.name as facility_name, CONCAT_WS(firstname, ' ', surname),
CASE WHEN b.memid = 0 THEN b.slots*f.guestcost 
ELSE b.slots*f.membercost END AS cost 
FROM Bookings as b
left join Facilities as f
on b.facid = f.facid
left join Members as m
on m.memid = b.memid
WHERE b.starttime like '2012-09-14%') as orig
Where cost>30
ORDER BY cost desc


In [6]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


In [1]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        select f.name as Facility, SUM(
    case 
        when b.memid = 0 then guestcost * slots 
        else membercost * slots
    end) as revenue
from Bookings as b
left join Facilities as f on f.facid = b.facid
group by f.name
having revenue < 1000
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Pool Table', 270)
('Snooker Table', 240)
('Table Tennis', 180)


In [3]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        select m.surname || ', ' || m.firstname as member_name, rb.surname || ', ' || rb.firstname as recommended_by
from Members as m
inner join Members as rb on m.recommendedby = rb.memid
order by member_name
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Bader, Florence', 'Stibbons, Ponder')
('Baker, Anne', 'Stibbons, Ponder')
('Baker, Timothy', 'Farrell, Jemima')
('Boothe, Tim', 'Rownam, Tim')
('Butters, Gerald', 'Smith, Darren')
('Coplin, Joan', 'Baker, Timothy')
('Crumpet, Erica', 'Smith, Tracy')
('Dare, Nancy', 'Joplette, Janice')
('Genting, Matthew', 'Butters, Gerald')
('Hunt, John', 'Purview, Millicent')
('Jones, David', 'Joplette, Janice')
('Jones, Douglas', 'Jones, David')
('Joplette, Janice', 'Smith, Darren')
('Mackenzie, Anna', 'Smith, Darren')
('Owen, Charles', 'Smith, Darren')
('Pinker, David', 'Farrell, Jemima')
('Purview, Millicent', 'Smith, Tracy')
('Rumney, Henrietta', 'Genting, Matthew')
('Sarwin, Ramnaresh', 'Bader, Florence')
('Smith, Jack', 'Smith, Darren')
('Stibbons, Ponder', 'Tracy, Burton')
('Worthington-Smyth, Henry', 'Smith, Tracy')


In [4]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        select f.name as facility, hours_booked
from Facilities as f
inner join (
    select facid, sum(slots)/2 as hours_booked from Bookings
    where memid > 0
    group by facid
    ) as member_bookings on member_bookings.facid = f.facid
order by hours_booked desc
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Badminton Court', 543)
('Tennis Court 1', 478)
('Massage Room 1', 442)
('Tennis Court 2', 441)
('Snooker Table', 430)
('Pool Table', 428)
('Table Tennis', 397)
('Squash Court', 209)
('Massage Room 2', 27)


In [5]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT f.name as facility, strftime('%m', starttime) as month, sum(slots)/2 as hours_booked
	FROM Bookings as b
    inner join Facilities as f on b.facid = f.facid
GROUP by month, facility
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Badminton Court', '07', 90)
('Massage Room 1', '07', 132)
('Massage Room 2', '07', 12)
('Pool Table', '07', 58)
('Snooker Table', '07', 78)
('Squash Court', '07', 82)
('Table Tennis', '07', 52)
('Tennis Court 1', '07', 135)
('Tennis Court 2', '07', 103)
('Badminton Court', '08', 229)
('Massage Room 1', '08', 246)
('Massage Room 2', '08', 41)
('Pool Table', '08', 161)
('Snooker Table', '08', 163)
('Squash Court', '08', 200)
('Table Tennis', '08', 152)
('Tennis Court 1', '08', 229)
('Tennis Court 2', '08', 241)
('Badminton Court', '09', 285)
('Massage Room 1', '09', 324)
('Massage Room 2', '09', 61)
('Pool Table', '09', 235)
('Snooker Table', '09', 213)
('Squash Court', '09', 270)
('Table Tennis', '09', 211)
('Tennis Court 1', '09', 295)
('Tennis Court 2', '09', 294)
